In [1]:
import os
import pandas as pd
import numpy as np

# 让 pandas 输出更舒服一点
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# 假设当前 notebook 路径: 根目录/notebooks/data_explore.ipynb
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
RAW_DIR = os.path.join(ROOT_DIR, "data_pipeline", "raw")
CONFIG_DIR = os.path.join(ROOT_DIR, "data_pipeline", "config")

RAW_DIR, CONFIG_DIR


('c:\\Users\\vflaz\\Desktop\\VF_Quant\\data_pipeline\\raw',
 'c:\\Users\\vflaz\\Desktop\\VF_Quant\\data_pipeline\\config')

In [2]:
tickers_path = os.path.join(CONFIG_DIR, "tickers.csv")
tickers_df = pd.read_csv(tickers_path)

# 只看一下
tickers_df


,ticker,source,description
0,SPY,yf,SPDR S&P 500 ETF Trust
1,XLK,yf,Technology Select Sector SPDR Fund
2,GLD,yf,SPDR Gold Shares
3,TLT,yf,iShares 20+ Year Treasury Bond ETF
4,SGOV,yf,iShares 0-3 Month Treasury Bond ETF


In [4]:
def load_raw_ticker(ticker: str) -> pd.DataFrame:
    """读取 raw/{ticker}.parquet 并做最小清洗（确保 date 是 datetime）"""
    path = os.path.join(RAW_DIR, f"{ticker}.parquet")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Raw file not found for {ticker}: {path}")
    
    df = pd.read_parquet(path)
    
    # 统一列名（应该已经是小写了，保险起见再转一次）
    df.columns = [str(c).lower() for c in df.columns]
    
    if "date" not in df.columns:
        raise ValueError(f"{ticker} parquet 中没有 'date' 列，当前列: {df.columns}")
    
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)
    return df

# 测试读取一个
spy_raw = load_raw_ticker("SPY")
spy_raw.head()


,date,adj_close,close,high,low,open,volume
0,1993-01-29,24.313028,43.93750,43.96875,43.75000,43.96875,1003200
1,1993-02-01,24.485958,44.25000,44.25000,43.96875,43.96875,480500
2,1993-02-02,24.537844,44.34375,44.37500,44.12500,44.21875,201300
3,1993-02-03,24.797228,44.81250,44.84375,44.37500,44.40625,529400
4,1993-02-04,24.900978,45.00000,45.09375,44.46875,44.96875,531500


In [5]:
summary_rows = []
tickers = tickers_df['ticker'].tolist()  # Extract tickers from the DataFrame

for ticker in tickers:
    df = load_raw_ticker(ticker)
    
    n_rows = len(df)
    min_date = df["date"].min()
    max_date = df["date"].max()
    
    # 关键字段的缺失情况（你可以关注 adj_close，或者 close）
    missing_adj = df["adj_close"].isna().sum() if "adj_close" in df.columns else None
    missing_close = df["close"].isna().sum() if "close" in df.columns else None
    missing_volume = df["volume"].isna().sum() if "volume" in df.columns else None
    
    summary_rows.append({
        "ticker": ticker,
        "rows": n_rows,
        "min_date": min_date,
        "max_date": max_date,
        "missing_adj_close": missing_adj,
        "missing_close": missing_close,
        "missing_volume": missing_volume,
    })

coverage_summary = pd.DataFrame(summary_rows)
coverage_summary


,ticker,rows,min_date,max_date,missing_adj_close,missing_close,missing_volume
0,SPY,8274,1993-01-29,2025-12-10,0,0,0
1,XLK,6784,1998-12-22,2025-12-10,0,0,0
2,GLD,5299,2004-11-18,2025-12-10,0,0,0
3,TLT,5881,2002-07-30,2025-12-10,0,0,0
4,SGOV,1391,2020-06-01,2025-12-10,0,0,0
